<a href="https://www.kaggle.com/code/abdelazizelsawy/a-b-test?scriptVersionId=99563508" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Analyze A/B Test Results 

This project will assure you have mastered the subjects covered in the statistics lessons. We have organized the current notebook into the following sections: 

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Conclusion](#conclusion)
- [Final Check](#finalcheck)
- [Submission](#submission)

Specific programming tasks are marked with a **ToDo** tag. 

<a id='intro'></a>
## Introduction

A/B tests are very commonly performed by data analysts and data scientists. For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should:
- Implement the new webpage, 
- Keep the old webpage, or 
- Perhaps run the experiment longer to make their decision.

Each **ToDo** task below has an associated quiz present in the classroom.  Though the classroom quizzes are **not necessary** to complete the project, they help ensure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the [rubric](https://review.udacity.com/#!/rubrics/1214/view) specification. 

>**Tip**: Though it's not a mandate, students can attempt the classroom quizzes to ensure statistical numeric values are calculated correctly in many cases.

<a id='probability'></a>
## Part I - Probability

To get started, let's import our libraries.

### ToDo 1.1
Now, read in the `ab_data.csv` data. Store it in `df`. Below is the description of the data, there are a total of 5 columns:

<center>

|Data columns|Purpose|Valid values|
| ------------- |:-------------| -----:|
|user_id|Unique ID|Int64 values|
|timestamp|Time stamp when the user visited the webpage|-|
|group|In the current A/B experiment, the users are categorized into two broad groups. <br>The `control` group users are expected to be served with `old_page`; and `treatment` group users are matched with the `new_page`. <br>However, **some inaccurate rows** are present in the initial data, such as a `control` group user is matched with a `new_page`. |`['control', 'treatment']`|
|landing_page|It denotes whether the user visited the old or new webpage.|`['old_page', 'new_page']`|
|converted|It denotes whether the user decided to pay for the company's product. Here, `1` means yes, the user bought the product.|`[0, 1]`|
</center>
Use your dataframe to answer the questions in Quiz 1 of the classroom.


>**Tip**: Please save your work regularly.

**a.** Read in the dataset from the `ab_data.csv` file and take a look at the top few rows here:

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

In [ ]:
df_abdata =  pd.read_csv ('../input/ab-data/ab_data.csv')
df_abdata.head()

the number of rows in the dataset.

In [ ]:
df_abdata.info()
df_abdata.shape

unique users in the dataset.

In [ ]:
unique_users=df_abdata['user_id'].unique()
print(len(unique_users))

The proportion of users converted.

In [ ]:
allcustomers = df_abdata['converted'].count()
buyers = df_abdata[df_abdata['converted']== 1].count()[0]
prop4buyers = buyers / allcustomers
print(round(prop4buyers,2))

The number of times when the "group" is treatment but "landing_page" is not a new_page.

In [ ]:
g_t = df_abdata['group'] =='treatment' 
l_n = df_abdata['landing_page']== 'new_page'
df_abdata.loc[g_t != l_n].count()[0]

Do any of the rows have missing values?

In [ ]:
null_values = df_abdata[df_abdata.isnull().any(axis=1)]
print(null_values)

### ToDo 1.2  
In a particular row, the **group** and **landing_page** columns should have either of the following acceptable values:

|user_id| timestamp|group|landing_page|converted|
|---|---|---|---|---|
|XXXX|XXXX|`control`| `old_page`|X |
|XXXX|XXXX|`treatment`|`new_page`|X |


It means, the `control` group users should match with `old_page`; and `treatment` group users should matched with the `new_page`. 

However, for the rows where `treatment` does not match with `new_page` or `control` does not match with `old_page`, we cannot be sure if such rows truly received the new or old wepage.  


Use **Quiz 2** in the classroom to figure out how should we handle the rows where the group and landing_page columns don't match?

**a.** Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
# Remove the inaccurate rows, and store the result in a new dataframe df2
df2= df_abdata.drop(df_abdata[((df_abdata['group'] == 'treatment') == (df_abdata['landing_page'] == 'new_page')) == False].index , inplace =True)
df2 = df_abdata

In [ ]:
# Double Check all of the incorrect rows were removed from df2 - 
# Output of the statement below should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

In [ ]:
df2.shape

### ToDo 1.3  
Use **df2** and the cells below to answer questions for **Quiz 3** in the classroom.

**a.** How many unique **user_id**s are in **df2**?

In [ ]:
unique_df2=df2['user_id'].unique()
print(len(unique_df2))

**b.** There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2.user_id.duplicated()]

**c.** Display the rows for the duplicate **user_id**? 

In [ ]:
df2.loc[(df2['user_id'] == 773192)]

In [ ]:
# Remove one of the rows with a duplicate user_id..
df2 = df2.drop(df2[(df2['user_id'] == 773192) & (df2['timestamp'] == '2017-01-09 05:37:58.781806')].index)

# Check again if the row with a duplicate user_id is deleted or not
df2[df2['user_id'] == 773192]

### ToDo 1.4  
Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

**a.** What is the probability of an individual converting regardless of the page they receive?<br><br>

>**Tip**: The probability  you'll compute represents the overall "converted" success rate in the population and you may call it $p_{population}$.



In [ ]:
new_customers = df2['converted'].count()
new_buyers = df2[df2['converted']== 1].count()[0]
df2_buyers =new_buyers / new_customers
print(round(df2_buyers,4))

**b.** Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
countCgroup = df2[df2['group'] == 'control'].count()[0]
df4cgroup = df2[df2['group'] == 'control']
prob4Cgroup = df4cgroup[df4cgroup['converted'] == 1].count()[0]/ countCgroup
print(round(prob4Cgroup,4))

**c.** Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
countTgroup = df2[df2['group'] == 'treatment'].count()[0]
df4tgroup = df2[df2['group'] == 'treatment']
prob4tgroup = df4tgroup[df4tgroup['converted'] == 1].count()[0]/ countTgroup
print(round(prob4tgroup,4))

In [ ]:
# Calculate the actual difference (obs_diff) between the conversion rates for the two groups.
obs_diff =prob4tgroup - prob4Cgroup  
print(round(obs_diff,4))

**d.** What is the probability that an individual received the new page?

In [ ]:
all_landing=df2['landing_page'].count()
landing_new =df2[df2['landing_page']=='new_page'].count()[0]
prob4landing_new = landing_new/all_landing
print(round(prob4landing_new,4))

**e.** Consider your results from parts (a) through (d) above, and explain below whether the new `treatment` group users lead to more conversions.

>**Your answer goes here.** the answer is for the new treatment group the probability = 0.1188 and the control group = 0.1204 which is higher than treatment group probability so there is no enough evidences that the new group will lead to more conversions 

In [ ]:
the_results_from_part = "The result of customers`s converting regardles the page = {} , in the control group = {} , in the treatment group = {} , and the individual`s receiving a new page = {}  .. based on that the gap between the control group and the treatment group is not that much so there is no enough evidences that supports (the new treatment group users lead to more conversions)    " .format(round(df2_buyers,4),round(prob4Cgroup,4),round(prob4tgroup,4),round(prob4landing_new,4))
print(the_results_from_part )

<a id='ab_test'></a>
## Part II - A/B Test

Since a timestamp is associated with each event, you could run a hypothesis test continuously as long as you observe the events. 

However, then the hard questions would be: 
- Do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  
- How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


### ToDo 2.1
For now, consider you need to make the decision just based on all the data provided.  

> Recall that you just calculated that the "converted" probability (or rate) for the old page is *slightly* higher than that of the new page (ToDo 1.4.c). 

If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should be your null and alternative hypotheses (**$H_0$** and **$H_1$**)?  

You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the "converted" probability (or rate) for the old and new pages respectively.

### ToDo 2.2 - Null Hypothesis $H_0$ Testing
Under the null hypothesis $H_0$, assume that $p_{new}$ and $p_{old}$ are equal. Furthermore, assume that $p_{new}$ and $p_{old}$ both are equal to the **converted** success rate in the `df2` data regardless of the page. So, our assumption is: <br><br>
<center>
$p_{new}$ = $p_{old}$ = $p_{population}$
</center>

In this section, you will: 

- Simulate (bootstrap) sample data set for both groups, and compute the  "converted" probability $p$ for those samples. 


- Use a sample size for each group equal to the ones in the `df2` data.


- Compute the difference in the "converted" probability for the two samples above. 


- Perform the sampling distribution for the "difference in the converted probability" between the two simulated-samples over 10,000 iterations; and calculate an estimate. 



Use the cells below to provide the necessary parts of this simulation.  You can use **Quiz 5** in the classroom to make sure you are on the right track.

In [ ]:
df2.head()

**a.** What is the **conversion rate** for $p_{new}$ under the null hypothesis? 

In [ ]:
P_new =df2.query("converted==1").shape[0]/df2.shape[0]
print(round(P_new,4))

**b.** What is the **conversion rate** for $p_{old}$ under the null hypothesis? 

In [ ]:
P_old =df2.query("converted==1").shape[0]/df2.shape[0]
print(round(P_old,4))

**c.** What is $n_{new}$, the number of individuals in the treatment group? <br><br>
*Hint*: The treatment group users are shown the new page.

In [ ]:
n_new =df2.query("landing_page == 'new_page'").count()[0]
print(n_new)

**d.** What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old =df2.query("landing_page == 'old_page'").count()[0]
print(n_old)

**e. Simulate Sample for the `treatment` Group**<br> 
Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null hypothesis.  <br><br>
*Hint*: Use `numpy.random.choice()` method to randomly generate $n_{new}$ number of values. <br>
Store these $n_{new}$ 1's and 0's in the `new_page_converted` numpy array.


In [ ]:
# Simulate a Sample for the treatment Group
new_page_converted = np.random.choice([0,1],n_new, p=(P_new,1-P_new))
new_page_converted

**f. Simulate Sample for the `control` Group** <br>
Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null hypothesis. <br> Store these $n_{old}$ 1's and 0's in the `old_page_converted` numpy array.

In [ ]:
# Simulate a Sample for the control Group
old_page_converted = np.random.choice([0,1],n_old, p=(P_old,1-P_old))
old_page_converted

**g.** Find the difference in the "converted" probability $(p{'}_{new}$ - $p{'}_{old})$ for your simulated samples from the parts (e) and (f) above. 

In [ ]:
new_page_converted.mean() - old_page_converted.mean()


**h. Sampling distribution** <br>
Re-create `new_page_converted` and `old_page_converted` and find the $(p{'}_{new}$ - $p{'}_{old})$ value 10,000 times using the same simulation process you used in parts (a) through (g) above. 

<br>
Store all  $(p{'}_{new}$ - $p{'}_{old})$  values in a NumPy array called `p_diffs`.

In [ ]:
#start time
from time import time
st=time()

# Sampling distribution 


p_diffs = []
size = df2.shape[0]
for _ in range (10000):
    new_page_converted = np.random.choice([0,1],n_new, p=(P_new,1-P_new))
    old_page_converted = np.random.choice([0,1],n_old, p=(P_old,1-P_old))
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())
    
#time taken
tt=time()-st
print(f"Loop Time taken: {tt} seconds")

#i feel like iam FREE .. thanks man .. NO MORE ROUNDED VALUES i got it


In [ ]:
plt.hist(p_diffs);
plt.axvline(x=obs_diff , color = 'r')
plt.title("The Difference Between New and Old page Rates")
plt.xlabel("Probabilities")
plt.ylabel("Frequency")
plt.axvline(np.percentile(p_diffs , 95) , color = 'g')

In [ ]:
p_diffs=np.array(p_diffs)

**j.** What proportion of the **p_diffs** are greater than the actual difference observed in the `df2` data?


In [ ]:
p = (p_diffs > obs_diff).mean()
print(round(p,4))

**k.** Please explain in words what you have just computed in part **j** above.  
 - What is this value called in scientific studies?  
 - What does this value signify in terms of whether or not there is a difference between the new and old pages? *Hint*: Compare the value above with the "Type I error rate (0.05)". 

>**Put your answer here.** I calculated the p-value if the null hypothesis is true then the actual difference observed in the df2 dataframe is (obs_diff) that we calculated earlier .... so the proportion of the p_diffs are greater than will be the mean of p_diffs > obs_diff .... p-value = 0.905



**l. Using Built-in Methods for Hypothesis Testing**<br>
We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. 

Fill in the statements below to calculate the:
- `convert_old`: number of conversions with the old_page
- `convert_new`: number of conversions with the new_page
- `n_old`: number of individuals who were shown the old_page
- `n_new`: number of individuals who were shown the new_page


In [ ]:
import statsmodels.api as sm

# number of conversions with the old_page
convert_old = df2.query("converted==1 & landing_page =='old_page'").shape[0]

# number of conversions with the new_page
convert_new =  df2.query("converted==1 & landing_page =='new_page'").shape[0]

# number of individuals who were shown the old_page
n_old = df2.query("landing_page =='old_page'").shape[0]

# number of individuals who received new_page
n_new = df2.query("landing_page =='new_page'").shape[0]



the_results_from_I = "The converted_old is {} , convert_new is {} , n_old is {} and n_new  is {} " .format(convert_old,convert_new,n_old,n_new)
print(the_results_from_I )

**m.** Now use `sm.stats.proportions_ztest()` to compute your test statistic and p-value.  [Here](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html) is a helpful link on using the built in.

The syntax is: 
```bash
proportions_ztest(count_array, nobs_array, alternative='larger')
```
where, 
- `count_array` = represents the number of "converted" for each group
- `nobs_array` = represents the total number of observations (rows) in each group
- `alternative` = choose one of the values from `[‘two-sided’, ‘smaller’, ‘larger’]` depending upon two-tailed, left-tailed, or right-tailed respectively. 
>**Hint**: <br>
It's a two-tailed if you defined $H_1$ as $(p_{new} = p_{old})$. <br>
It's a left-tailed if you defined $H_1$ as $(p_{new} < p_{old})$. <br>
It's a right-tailed if you defined $H_1$ as $(p_{new} > p_{old})$. 

The built-in function above will return the z_score, p_value. 

---
### About the two-sample z-test
Recall that you have plotted a distribution `p_diffs` representing the
difference in the "converted" probability  $(p{'}_{new}-p{'}_{old})$  for your two simulated samples 10,000 times. 

Another way for comparing the mean of two independent and normal distribution is a **two-sample z-test**. You can perform the Z-test to calculate the Z_score, as shown in the equation below:

$$
Z_{score} = \frac{ (p{'}_{new}-p{'}_{old}) - (p_{new}  -  p_{old})}{ \sqrt{ \frac{\sigma^{2}_{new} }{n_{new}} + \frac{\sigma^{2}_{old} }{n_{old}}  } }
$$

where,
- $p{'}$ is the "converted" success rate in the sample
- $p_{new}$ and $p_{old}$ are the "converted" success rate for the two groups in the population. 
- $\sigma_{new}$ and $\sigma_{new}$ are the standard deviation for the two groups in the population. 
- $n_{new}$ and $n_{old}$ represent the size of the two groups or samples (it's same in our case)


>Z-test is performed when the sample size is large, and the population variance is known. The z-score represents the distance between the two "converted" success rates in terms of the standard error. 

Next step is to make a decision to reject or fail to reject the null hypothesis based on comparing these two values: 
- $Z_{score}$
- $Z_{\alpha}$ or $Z_{0.05}$, also known as critical value at 95% confidence interval.  $Z_{0.05}$ is 1.645 for one-tailed tests,  and 1.960 for two-tailed test. You can determine the $Z_{\alpha}$ from the z-table manually. 

Decide if your hypothesis is either a two-tailed, left-tailed, or right-tailed test. Accordingly, reject OR fail to reject the  null based on the comparison between $Z_{score}$ and $Z_{\alpha}$. We determine whether or not the $Z_{score}$ lies in the "rejection region" in the distribution. In other words, a "rejection region" is an interval where the null hypothesis is rejected iff the $Z_{score}$ lies in that region.

>Hint:<br>
For a right-tailed test, reject null if $Z_{score}$ > $Z_{\alpha}$. <br>
For a left-tailed test, reject null if $Z_{score}$ < $Z_{\alpha}$. 



In [ ]:
import statsmodels.api as sm
# ToDo: Complete the sm.stats.proportions_ztest() method arguments
count_array = np.array([convert_new, convert_old])
nobs_array = np.array([n_old, n_new])


z_score, p_value = sm.stats.proportions_ztest([convert_new, convert_old], [n_new, n_old], alternative='two-sided')

print("this is z_score {}".format(z_score))
print("this is p_value {}".format(p_value))


**n.** What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?<br><br>

>**Tip**: Notice whether the p-value is similar to the one computed earlier. Accordingly, can you reject/fail to reject the null hypothesis? It is important to correctly interpret the test statistic and p-value.

z-score = (obs_diff 2) standard deviation .......... z-score and p-value values agree with what we got in the previous question in j and k p_value = 0.18988337448195103 ................. we can not reject our null hypothesis<a id='regression'></a>
### Part III - A regression approach

### ToDo 3.1 
In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

**a.** Since each row in the `df2` data is either a conversion or no conversion, what type of regression should you be performing in this case?

<a id='regression'></a>
### Part III - A regression approach

### ToDo 3.1 
In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

**a.** Since each row in the `df2` data is either a conversion or no conversion, what type of regression should you be performing in this case?

Since we are dealing with binary outputs , the type of regression is logistic regression

**b.** The goal is to use **statsmodels** library to fit the regression model you specified in part **a.** above to see if there is a significant difference in conversion based on the page-type a customer receives. However, you first need to create the following two columns in the `df2` dataframe:
 1. `intercept` - It should be `1` in the entire column. 
 2. `ab_page` - It's a dummy variable column, having a value `1` when an individual receives the **treatment**, otherwise `0`.  

In [ ]:
df2['intercept'] =1

In [ ]:
df2.head()

In [ ]:
df2= df2.join(pd.get_dummies(df2['landing_page']))

In [ ]:
df2['ab_page'] = pd.get_dummies(df2['group']) ['treatment']
df2.head()


**c.** Use **statsmodels** to instantiate your regression model on the two columns you created in part (b). above, then fit the model to predict whether or not an individual converts. 


In [ ]:
f_model = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
result4f_m = f_model.fit()

**d.** Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
result4f_m.summary2()

**e.** What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  


*we have two different values in part2 and 3 because in part 2 it was 1 tail and in part3 the linear regression depend on two-sided test.
*P-value is still greater that error rate so we can not reject the null hypothesi.
*p_value = 0.1899.

**f.** Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

*The more we get the more we give ..... if we get extra informations like Age , social statu , education level , jop title or total income .. all of the above will definitely have an effect an indiviual converts.

**The more we get the more we give ..... if we get extra informations like Age , social statu , education level , jop title or total income .. all of the above will definitely have an effect an indiviual converts.

**g. Adding countries**<br> 
Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. 

1. You will need to read in the **countries.csv** dataset and merge together your `df2` datasets on the appropriate rows. You call the resulting dataframe `df_merged`. [Here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) are the docs for joining tables. 

2. Does it appear that country had an impact on conversion?  To answer this question, consider the three unique values, `['UK', 'US', 'CA']`, in the `country` column. Create dummy variables for these country columns. 
>**Hint:** Use `pandas.get_dummies()` to create dummy variables. **You will utilize two columns for the three dummy variables.** 

 Provide the statistical output as well as a written response to answer this question.

In [ ]:
# Read the countries.csv
countries=pd.read_csv('../input/countries/countries4abtest.csv')
countries.head()

In [ ]:
# Join with the df2 dataframe
df3 = countries.set_index('user_id').join(df2.set_index('user_id'),how ='inner')

In [ ]:
df3['intercept'] =1

In [ ]:
pd.get_dummies(df3['country']).head()

In [ ]:
# Create the necessary dummy variables
df3[['UK','US','CA']]=pd.get_dummies(df3['country'])
df3.head()

In [ ]:
df3['US_inter'] = df3['US'] * df3['ab_page']
df3['UK_inter'] = df3['UK'] * df3['ab_page']

In [ ]:
p4usconverted=df3.query("converted==1 & country=='US'").shape[0]/df3['converted'].count()
p4ukconverted=df3.query("converted==1 & country=='UK'").shape[0]/df3['converted'].count()
p4caconverted=df3.query("converted==1 & country=='CA'").shape[0]/df3['converted'].count()

print("The convertion for US is {} , for UK {} and for CA {} ".format(round(p4usconverted,4) , round(p4ukconverted,4) , round(p4caconverted,4)))


**h. Fit your model and obtain the results**<br> 
Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if are there significant effects on conversion.  **Create the necessary additional columns, and fit the new model.** 


Provide the summary results (statistical output), and your conclusions (written response) based on the results. 


In [ ]:
# Fit your model, and summarize the results
f_model = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
result4f_m = f_model.fit()

In [ ]:
_2nd_model = sm.Logit(df3['converted'], df3[['intercept', 'US', 'UK' ,'ab_page' , 'US_inter', 'UK_inter']])
result4_2nd_m = _2nd_model.fit()

In [ ]:
result4_2nd_m.summary2()

<a id='conclusion'></a>
## Conclusion

>**conclusion.** we don't have enough evidence that supports the new page is better than the old one .. I vote for the old page